### Exercise 4 - Document clustering, and introduction to N-grams

In [368]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
import numpy as np
import scipy
from numpy import matlib

In [417]:
def basicfilecrawler(file_path):
    # Store filenames read and their text content
    #num_files_read=0
    #crawled_filenames=[]
    #crawled_texts=[]
    #directory_contentlists=gettextlist(directory_path)
    # In this basic crawled we just process text files
    # and do not handle subdirectories
    #directory_textfiles=directory_contentlists[0]
    
    temp_file=open(file_path,'r',encoding='utf-8',errors='ignore')
    temp_text=temp_file.read() 
    mytext_paragraphs=re.split(r'\n[ \n]*\n', temp_text) 
    temp_file.close()
    
    return(mytext_paragraphs)
    

In [418]:
mytext_paragraphs = basicfilecrawler("the _wonderful _wizard.txt")

In [420]:
len(mytext_paragraphs)

1155

In [421]:
# Tokenize downloaded texts paragraph by paragraph
# and change them to NLTK format
mydownloaded_nltk_texts = []
for k in range(len(mytext_paragraphs)):
    temp_tokenizedtext = nltk.word_tokenize(mytext_paragraphs[k])
    temp_nltktext = nltk.Text(temp_tokenizedtext)
    mydownloaded_nltk_texts.append(temp_nltktext)

In [422]:
mydownloaded_nltk_texts[0]

<Text: Dorothy lived in the midst of the great...>

In [423]:
# Make all downloaded texts lowercase
mydownloaded_lowercase_texts = []
for k in range(len(mydownloaded_nltk_texts)):
    temp_lowercase_text = []
    for l in range(len(mydownloaded_nltk_texts[k])):
        lowercase_word = mydownloaded_nltk_texts[k][l].lower()
        temp_lowercase_text.append(lowercase_word)
    temp_lowercasetest = nltk.Text(temp_lowercase_text)
    mydownloaded_lowercase_texts.append(temp_lowercase_text)

In [424]:
# Convert a POS tag for WordNet
def tagtowordnet(postag):
    wordnettag=-1
    if postag[0]=='N':
        wordnettag='n'
    elif postag[0]=='V':
        wordnettag='v'
    elif postag[0]=='J':
        wordnettag='a'
    elif postag[0]=='R':
        wordnettag='r'
    return(wordnettag)

In [425]:
# POS tag and lemmatize the loaded texts
# Download tagger and wordnet resources if you do not have them already
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# lemmatize downloaded texts
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatizetext(nltktexttolemmatize):
    # tag the text with POS tags
    taggedtext = nltk.pos_tag(nltktexttolemmatize)
    # lemmatize each word text
    lemmatizedtext = []
    for l in range(len(taggedtext)):
        # Lemmatize a word using the WordNet converted POS tag
        wordtolemmatize = taggedtext[l][0]
        wordnettag = tagtowordnet(taggedtext[l][1])
        if wordnettag != -1:
            lemmatizedword = lemmatizer.lemmatize(wordtolemmatize, wordnettag)
        else:
            lemmatizedword = wordtolemmatize
        # store the lemmatized word
        lemmatizedtext.append(lemmatizedword)
    return(lemmatizedtext)



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tunde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tunde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [426]:
mydownloaded_lemmatizedtexts = []
for k in range(len(mydownloaded_lowercase_texts)):
    lemmatizedtext = lemmatizetext(mydownloaded_lowercase_texts[k])
    lemmatizedtext = nltk.Text(lemmatizedtext)
    mydownloaded_lemmatizedtexts.append(lemmatizedtext)

In [427]:
mydownloaded_lemmatizedtexts[-1]

<Text: `` from the land of oz , ''...>

In [428]:
# find the vocabularies of each document/paragraph
myvocabularies = []
myindices_in_vocabularies = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # get unique words and where they occur
    temptext = mydownloaded_lemmatizedtexts[k]
    unique_results = np.unique(temptext, return_inverse = True)
    unique_words = unique_results[0]
    word_indices = unique_results[1]
    # store the vocabularies and the indices
    myvocabularies.append(unique_words)
    myindices_in_vocabularies.append(word_indices)

In [429]:
len(myvocabularies)

1155

In [430]:
len(myindices_in_vocabularies)

1155

In [431]:
# unify the vocabularies
# first concatenate all vocabularies
tempvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])

# find unique words among all the vocabularies
uniqueresults = np.unique(tempvocabulary, return_inverse = True)
unifiedvocabulary = uniqueresults[0]
wordindices = uniqueresults[1]

In [432]:
len(unifiedvocabulary)

2265

In [433]:
wordindices

array([  14,   23,   24, ..., 2015, 2024, 2028], dtype=int64)

In [434]:
# translate previous indices to the unified vocabulary
# must keep track where each vocabulary started in the concatenated one
vocabularystart = 0
myindices_in_unifiedvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # in order to shift word indices, we must temporarily
    # convert their data type to a numpy array
    tempindices = np.array(myindices_in_vocabularies[k])
    tempindices = tempindices + vocabularystart
    tempindices = wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart = vocabularystart + len(myvocabularies[k])

In [435]:
len(myindices_in_unifiedvocabulary)

1155

In [436]:
# total count of each unique word over all the downloaded documents
unifiedvocabulary_totaloccurrencecounts=np.zeros((len(unifiedvocabulary),1))

# count occurrences
for k in range(len(mydownloaded_lemmatizedtexts)):
    occurrencecounts = np.zeros((len(unifiedvocabulary), 1))
    for l in range(len(myindices_in_unifiedvocabulary[k])):
        occurrencecounts[myindices_in_unifiedvocabulary[k][l]] = (occurrencecounts[myindices_in_unifiedvocabulary[k][l]]
                                                                 + 1)
    unifiedvocabulary_totaloccurrencecounts = unifiedvocabulary_totaloccurrencecounts + occurrencecounts

In [437]:
len(unifiedvocabulary_totaloccurrencecounts)

2265

In [438]:
# top-100 words according to the largest total occurrence count
highest_totaloccurrences_indices = np.argsort(-1*unifiedvocabulary_totaloccurrencecounts, axis=0)
print(np.squeeze(unifiedvocabulary[highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(unifiedvocabulary_totaloccurrencecounts[highest_totaloccurrences_indices[0:100]]))

['the' ',' '.' 'and' 'be' 'to' "''" '``' 'of' 'a' 'i' 'you' 'in' 'have'
 'he' 'it' 'her' 'they' 'she' 'that' 'dorothy' 'say' 'as' 'for' ';' 'so'
 'but' 'do' 'not' 'with' 'at' 'all' 'them' 'scarecrow' 'his' 'me' '?' 'my'
 'him' 'come' 'woodman' 'lion' 'when' 'will' 'on' 'go' 'oz' 'we' 'make'
 'great' 'this' 'ask' 'tin' 'witch' 'if' 'little' 'there' 'one' 'then'
 'from' 'could' 'see' 'who' 'would' 'no' 'out' 'get' 'up' 'green' 'can'
 'head' 'look' 'their' 'back' '!' 'down' 'girl' 'know' 'toto' 'over'
 'think' 'by' 'what' 'answer' 'again' 'upon' 'good' 'very' 'where' 'give'
 'shall' 'now' "n't" 'find' 'city' 'into' "'s" 'must' 'man' 'walk']
[2934. 2719. 1850. 1668. 1476. 1109. 1086. 1079.  826.  802.  649.  496.
  480.  479.  456.  427.  411.  402.  400.  394.  360.  359.  329.  323.
  321.  308.  302.  283.  274.  272.  254.  246.  238.  226.  216.  198.
  194.  189.  184.  182.  181.  180.  160.  159.  158.  156.  155.  150.
  149.  147.  147.  139.  138.  138.  137.  136.  129.  128.  

In [439]:
#%% Vocabulary pruning
nltkstopwords = nltk.corpus.stopwords.words('english')
pruningdecisions = np.zeros((len(unifiedvocabulary),1))
for k in range(len(unifiedvocabulary)):
    # Rule 1: check the nltk stop word list
    if (unifiedvocabulary[k] in nltkstopwords):
        pruningdecisions[k] = 1
    # Rule 2: if the word is too short
    if (len(unifiedvocabulary[k]) < 2):
        pruningdecisions[k] = 1
      # Rule 3: if the word is too long
    if (len(unifiedvocabulary[k]) > 20):
        pruningdecisions[k] = 1
    # Rule 4: if the word occurs less than 4 times
    if(unifiedvocabulary_totaloccurrencecounts[k] < 4):
        pruningdecisions[k] = 1

In [440]:
print('Top-100 words after pruning the unified vocabulary:\n')
remaining_indices = np.squeeze(np.where(pruningdecisions==0)[0])
remaining_vocabulary = unifiedvocabulary[remaining_indices]
remainingvocabulary_totaloccurrencecounts = unifiedvocabulary_totaloccurrencecounts[remaining_indices]
remaining_highest_totaloccurrences_indices = np.argsort(-1*remainingvocabulary_totaloccurrencecounts, axis=0)
print(np.squeeze(remaining_vocabulary[remaining_highest_totaloccurrences_indices[0:100]]))
print(np.squeeze(remainingvocabulary_totaloccurrencecounts[remaining_highest_totaloccurrences_indices[0:100]]))

Top-100 words after pruning the unified vocabulary:

["''" '``' 'dorothy' 'say' 'scarecrow' 'come' 'woodman' 'lion' 'go' 'oz'
 'make' 'great' 'ask' 'witch' 'tin' 'little' 'one' 'could' 'see' 'would'
 'get' 'green' 'head' 'look' 'back' 'know' 'girl' 'toto' 'think' 'answer'
 'upon' 'good' 'give' 'shall' "n't" 'find' 'city' "'s" 'man' 'must'
 'wicked' 'walk' 'emerald' 'long' 'well' 'heart' 'take' 'room' 'away' 'us'
 'country' 'like' 'way' 'big' 'time' 'carry' 'tree' 'people' 'tell' 'saw'
 'brain' 'never' 'eye' 'reply' 'monkey' 'stand' 'many' 'day' 'live'
 'first' 'run' 'help' 'road' 'ever' 'forest' 'friend' 'soon' 'house'
 'keep' 'much' 'beast' 'wish' 'arm' 'around' 'sit' 'cry' 'wizard' 'thing'
 'call' 'beautiful' 'old' 'oh' 'land' 'fly' 'shoe' 'woman' 'air' 'kill'
 'kansas' 'quite']
[1086. 1079.  360.  359.  226.  182.  181.  180.  156.  155.  149.  147.
  139.  138.  138.  136.  128.  122.  122.  113.  107.  105.  102.  101.
  100.   96.   96.   93.   90.   87.   85.   85.   83.   82.  

In [441]:
#%% Get indices of documents to remaining words
oldtopruned=[]
tempind=-1
for k in range(len(unifiedvocabulary)):
    if pruningdecisions[k]==0:
        tempind=tempind+1
        oldtopruned.append(tempind)
    else:
        oldtopruned.append(-1)

In [442]:
#%% Create pruned texts
mycrawled_prunedtexts=[]
myindices_in_prunedvocabulary=[]
for k in range(len(mydownloaded_lemmatizedtexts)):
    #print(k)
    temp_newindices=[]
    temp_newdoc=[]
    for l in range(len(mydownloaded_lemmatizedtexts[k])):
        temp_oldindex=myindices_in_unifiedvocabulary[k][l]
        temp_newindex=oldtopruned[temp_oldindex]
        if temp_newindex!=-1:
            temp_newindices.append(temp_newindex)
            temp_newdoc.append(unifiedvocabulary[temp_oldindex])
    mycrawled_prunedtexts.append(temp_newdoc)
    myindices_in_prunedvocabulary.append(temp_newindices)


In [443]:
len(remaining_vocabulary)

802

In [444]:
# create count vectors
n_docs = len(mycrawled_prunedtexts) # 1449=total number of documents
n_vocab = len(remaining_vocabulary) # 920
# matrix of term frequencies
tfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab)) # 1449x920
# row vector of document frequencies
dfvector = scipy.sparse.lil_matrix((1, n_vocab)) #1x920

In [445]:
len(mycrawled_prunedtexts[0])

70

In [446]:
# loop over the documents
for k in range(n_docs):
    # row vector of which words occur in this document
    temp_dfvector = scipy.sparse.lil_matrix((1, n_vocab)) # 1x920
    # we loop over the words in each document
    for l in range(len(mycrawled_prunedtexts[k])):
        # add current word to term-frequency count and document count
        currentword = myindices_in_prunedvocabulary[k][l]
        tfmatrix[k, currentword] = tfmatrix[k, currentword] + 1
        #tfmatrix[k, currentword] = tfmatrix[k, currentword]/len(mycrawled_prunedtexts[k])
        temp_dfvector[0, currentword] = 1
    # add which words occur in this document to overall document counts
    
    dfvector = dfvector + temp_dfvector

In [447]:
tfmatrix.data

array([list([1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0]),
       list([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0]),
       list([1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
       ..., list([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
       list([2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
       list([2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])],
      dtype=object)

In [448]:
# length-normalized frequency for the TF part
for i in range(n_docs):
    for j in range(len(tfmatrix.data[i])):
        tfmatrix.data[i][j] = tfmatrix.data[i][j]/len(tfmatrix.data[i])

In [450]:
# create IDF and TF-IDF vectors
# smoothed logarithmic idf
idfvector = np.squeeze(np.array(dfvector.todense()))
idfvector = np.log(1 + ((idfvector+1)**-1)*n_docs)

In [451]:
# use the count statistics to compute the tf-idf matrix
tfidfmatrix = scipy.sparse.lil_matrix((n_docs, n_vocab))
# use the count statistics to compute the tf-idf matrix
for k in range(n_docs):
    # find nonzero term frequencies
    tempindices = np.nonzero(tfmatrix[k, :])[1]
    tfterm = np.squeeze(np.array(tfmatrix[k, tempindices].todense()))
    tfidfmatrix[k, tempindices] = tfterm * idfvector[tempindices]

In [452]:
#%% Use the TF-IDF matrix as data to be clustered
X=tfidfmatrix
# Normalize the documents to unit vector norm
tempnorms=np.squeeze(np.array(np.sum(X.multiply(X),axis=1)))
# If any documents have zero norm, avoid dividing them by zero
tempnorms[tempnorms==0]=1
X=scipy.sparse.diags(tempnorms**-0.5).dot(X)
n_data=np.shape(X)[0]
n_dimensions=np.shape(X)[1]

In [453]:
#%% Initialize the Gaussian mixture model
# Function to initialize the Gaussian mixture model, create component parameters
def initialize_mixturemodel(X,n_components):
    # Create lists of sparse matrices to hold the parameters
    n_dimensions=np.shape(X)[1]
    n_data = np.shape(X)[0]
    mixturemodel_means=scipy.sparse.lil_matrix((n_components,n_dimensions))
    mixturemodel_weights=np.zeros((n_components))
    mixturemodel_covariances=[]
    mixturemodel_inversecovariances=[]
    for k in range(n_components):
        tempcovariance=scipy.sparse.lil_matrix((n_dimensions,n_dimensions))
        mixturemodel_covariances.append(tempcovariance)
        tempinvcovariance=scipy.sparse.lil_matrix((n_dimensions,n_dimensions))
        mixturemodel_inversecovariances.append(tempinvcovariance)
    # Initialize the parameters
    for k in range(n_components):
        mixturemodel_weights[k]=1/n_components
        # Pick a random data point as the initial mean
        tempindex=scipy.stats.randint.rvs(low=0,high=n_data)
        mixturemodel_means[k]=X[tempindex,:].toarray()
        # Initialize the covariance matrix to be spherical
        for l in range(n_dimensions):
            mixturemodel_covariances[k][l,l]=1
            mixturemodel_inversecovariances[k][l,l]=1
    return(mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,mixturemodel_inversecovariances)

In [454]:
def run_estep(X,mixturemodel_means,mixturemodel_covariances,mixturemodel_inversecovariances,mixturemodel_weights):
    # For each component, compute terms that do not involve data
    meanterms=np.zeros((n_components))
    logdeterminants=np.zeros((n_components))
    logconstantterms=np.zeros((n_components))
    
    for k in range(n_components):
        # Compute mu_k*inv(Sigma_k)*mu_k
        meanterms[k]=(mixturemodel_means[k,:]*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T)[0,0]
        # Compute determinant of Sigma_k. For a diagonal matrix
        # this is just the product of the main diagonal
        logdeterminants[k]=np.sum(np.log(mixturemodel_covariances[k].diagonal(0)))
        # Compute constant term beta_k * 1/(|Sigma_k|^1/2)
        # Omit the (2pi)^d/2 as it cancels out
        logconstantterms[k]=np.log(mixturemodel_weights[k]) - 0.5*logdeterminants[k]
    
    print('E-step part2 ')
    # Compute terms that involve distances of data from components
    xnorms=np.zeros((n_data,n_components))
    xtimesmu=np.zeros((n_data,n_components))
    
    for k in range(n_components):
        #print(k)
        xnorms[:,k]=(X*mixturemodel_inversecovariances[k]*X.T).diagonal(0)
        xtimesmu[:,k]=np.squeeze((X*mixturemodel_inversecovariances[k]* mixturemodel_means[k,:].T).toarray())
        
    xdists=xnorms+np.matlib.repmat(meanterms,n_data,1)-2*xtimesmu
    # Substract maximal term before exponent (cancels out) to maintain computational precision
    numeratorterms=logconstantterms-xdists/2
    numeratorterms-=np.matlib.repmat(np.max(numeratorterms,axis=1),n_components,1).T
    numeratorterms=np.exp(numeratorterms)
    mixturemodel_componentmemberships=numeratorterms/np.matlib.repmat(np.sum(numeratorterms,axis=1),n_components,1).T
    return(mixturemodel_componentmemberships)

In [455]:
def run_mstep_sumweights(mixturemodel_componentmemberships):
    # Compute total weight per component
    mixturemodel_weights=np.sum(mixturemodel_componentmemberships,axis=0)
    return(mixturemodel_weights)

def run_mstep_means(X,mixturemodel_componentmemberships,mixturemodel_weights):
    # Update component means
    mixturemodel_means=scipy.sparse.lil_matrix((n_components,n_dimensions))
    for k in range(n_components):
        mixturemodel_means[k,:]=np.sum(scipy.sparse.diags(mixturemodel_componentmemberships[:,k]).dot(X),axis=0)
        mixturemodel_means[k,:]/=mixturemodel_weights[k]
    return(mixturemodel_means)

def run_mstep_covariances(X,mixturemodel_componentmemberships,mixturemodel_weights,mixturemodel_means):
    # Update diagonal component covariance matrices
    n_dimensions=np.shape(X)[1]
    n_components=np.shape(mixturemodel_componentmemberships)[1]
    tempcovariances=np.zeros((n_components,n_dimensions))
    mixturemodel_covariances=[]
    mixturemodel_inversecovariances=[]
    
    for k in range(n_components):
        tempcovariances[k,:]= np.sum(scipy.sparse.diags(
            mixturemodel_componentmemberships[:,k]).dot(
            X.multiply(X)),axis=0)-mixturemodel_means[k,:].multiply(mixturemodel_means[k,:])*mixturemodel_weights[k]
        tempcovariances[k,:]/=mixturemodel_weights[k]
        # Convert to sparse matrices
        tempepsilon=1e-10
        # Add a small regularization term
        temp_covariance=scipy.sparse.diags(tempcovariances[k,:]+tempepsilon)
        temp_inversecovariance=scipy.sparse.diags((tempcovariances[k,:]+tempepsilon)**-1)
        mixturemodel_covariances.append(temp_covariance)
        mixturemodel_inversecovariances.append(temp_inversecovariance)
    return(mixturemodel_covariances,mixturemodel_inversecovariances)

def run_mstep_normalizeweights(mixturemodel_weights):
    # Update mixture-component prior probabilities
    mixturemodel_weights/=sum(mixturemodel_weights)
    return(mixturemodel_weights)

In [459]:
#%% Perform the EM algorithm iterations
def perform_emalgorithm(X,n_components,n_emiterations):
    mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,\
 mixturemodel_inversecovariances=initialize_mixturemodel(X,n_components)

    for t in range(n_emiterations):
        # ====== E-step: Compute the component membership
        # probabilities of each data point ======
        print('E-step ' + str(t))

        mixturemodel_componentmemberships=run_estep(X,mixturemodel_means,mixturemodel_covariances,
                                            mixturemodel_inversecovariances,mixturemodel_weights)
        # ====== M-step: update component parameters======
        print('M-step ' + str(t))
        print('M-step part1 ' + str(t))
        mixturemodel_weights=run_mstep_sumweights(mixturemodel_componentmemberships)
        print('M-step part2 ' + str(t))

        mixturemodel_means=run_mstep_means(X,mixturemodel_componentmemberships,mixturemodel_weights)
        print('M-step part3 ' + str(t))
        mixturemodel_covariances,mixturemodel_inversecovariances=run_mstep_covariances(
            X,mixturemodel_componentmemberships,mixturemodel_weights,mixturemodel_means)
        
        print('M-step part4 ' + str(t))
        mixturemodel_weights=run_mstep_normalizeweights(mixturemodel_weights)
    return(mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,mixturemodel_inversecovariances)


In [460]:
# Try out the functions we just defined on the data
n_components=10
n_emiterations=100
mixturemodel_weights,mixturemodel_means,mixturemodel_covariances,mixturemodel_inversecovariances = perform_emalgorithm(
    X,n_components,n_emiterations)

E-step 0
E-step part2 
M-step 0
M-step part1 0
M-step part2 0
M-step part3 0
M-step part4 0
E-step 1
E-step part2 
M-step 1
M-step part1 1
M-step part2 1
M-step part3 1
M-step part4 1
E-step 2
E-step part2 
M-step 2
M-step part1 2
M-step part2 2
M-step part3 2
M-step part4 2
E-step 3
E-step part2 
M-step 3
M-step part1 3
M-step part2 3
M-step part3 3
M-step part4 3
E-step 4
E-step part2 
M-step 4
M-step part1 4
M-step part2 4
M-step part3 4
M-step part4 4
E-step 5
E-step part2 
M-step 5
M-step part1 5
M-step part2 5
M-step part3 5
M-step part4 5
E-step 6
E-step part2 
M-step 6
M-step part1 6
M-step part2 6
M-step part3 6
M-step part4 6
E-step 7
E-step part2 
M-step 7
M-step part1 7
M-step part2 7
M-step part3 7
M-step part4 7
E-step 8
E-step part2 
M-step 8
M-step part1 8
M-step part2 8
M-step part3 8
M-step part4 8
E-step 9
E-step part2 
M-step 9
M-step part1 9
M-step part2 9
M-step part3 9
M-step part4 9
E-step 10
E-step part2 
M-step 10
M-step part1 10
M-step part2 10
M-step part3 1

M-step 84
M-step part1 84
M-step part2 84
M-step part3 84
M-step part4 84
E-step 85
E-step part2 
M-step 85
M-step part1 85
M-step part2 85
M-step part3 85
M-step part4 85
E-step 86
E-step part2 
M-step 86
M-step part1 86
M-step part2 86
M-step part3 86
M-step part4 86
E-step 87
E-step part2 
M-step 87
M-step part1 87
M-step part2 87
M-step part3 87
M-step part4 87
E-step 88
E-step part2 
M-step 88
M-step part1 88
M-step part2 88
M-step part3 88
M-step part4 88
E-step 89
E-step part2 
M-step 89
M-step part1 89
M-step part2 89
M-step part3 89
M-step part4 89
E-step 90
E-step part2 
M-step 90
M-step part1 90
M-step part2 90
M-step part3 90
M-step part4 90
E-step 91
E-step part2 
M-step 91
M-step part1 91
M-step part2 91
M-step part3 91
M-step part4 91
E-step 92
E-step part2 
M-step 92
M-step part1 92
M-step part2 92
M-step part3 92
M-step part4 92
E-step 93
E-step part2 
M-step 93
M-step part1 93
M-step part2 93
M-step part3 93
M-step part4 93
E-step 94
E-step part2 
M-step 94
M-step par

In [466]:
# Find top 10 words for each cluster
for k in range(n_components):
    print(k)
    highest_dimensionweight_indices=np.argsort(-np.squeeze(mixturemodel_means[k,:].toarray()),axis=0)

    print(' '.join(remaining_vocabulary[highest_dimensionweight_indices[1:10]]))

0
'' winged `` command three cap time say bow
1
`` answer try lion one say come woodman give
2
tin '' `` scarecrow say 've axe lion --
3
`` ask scarecrow dorothy say go woodman tin brain
4
tree dorothy look come scarecrow long house wait sit
5
'' ask dorothy say scarecrow man oz get answer
6
soldier gate dorothy girl whisker dress little city emerald
7
`` lion courage say cowardly give oz woodman promise
8
wicked winkies `` '' west east tell silver country
9
`` say dorothy oz n't answer great go must


In [464]:
# Version 2 - Get documents closest to component mean, i.e. highest p(d|k).
# ---The computation of distances here is the same as done in the E-step of EM---
# For each component, compute terms that do not involve data
meanterms=np.zeros((n_components))
logdeterminants=np.zeros((n_components))
logconstantterms=np.zeros((n_components))
for k in range(n_components):
    # Compute mu_k*inv(Sigma_k)*mu_k
    meanterms[k]=(mixturemodel_means[k,:]*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T)[0,0]

# Compute terms that involve distances of data from components
xnorms=np.zeros((n_data,n_components))
xtimesmu=np.zeros((n_data,n_components))

for k in range(n_components):
    xnorms[:,k]=(X*mixturemodel_inversecovariances[k]*X.T).diagonal(0)
    xtimesmu[:,k]=np.squeeze((X*mixturemodel_inversecovariances[k]*mixturemodel_means[k,:].T).toarray())

xdists=xnorms+np.matlib.repmat(meanterms,n_data,1)-2*xtimesmu

for k in range(n_components):
    tempdists=np.array(np.squeeze(xdists[:,k]))
    highest_componentprob_indices=np.argsort(-1*tempdists,axis=0)
    print(k)
    print(highest_componentprob_indices[0:10])
    print(' '.join(mydownloaded_nltk_texts[highest_componentprob_indices[0]]))

0
[ 815  803  886  883  798  656  260  797  359 1145]
So they sat down and listened while he told the following tale :
1
[ 797  945  954  883  101  733   67 1130  412  798]
`` Not a bit of it , my dear ; I 'm just a common man . ''
2
[803 683 100 883 469 101 733  67 738 798]
`` Exactly so ! I am a humbug . ''
3
[ 886  556  816  883  101  733   67  195  916 1037]
The Lion hesitated no longer , but drank till the dish was empty .
4
[803 733 412 798 955 813 756 216 569 894]
`` Exactly so ! I am a humbug . ''
5
[ 589  980  803  864  556  990  816 1068 1024  306]
[ Illustration : `` _The Soldier with the green whiskers led them through the streets._ '' ]
6
[ 883  733  798 1006  306  571  756  995  216 1021]
`` Drink . ''
7
[ 803  954 1024  101  733   67  195  738  798 1101]
`` Exactly so ! I am a humbug . ''
8
[1028  886  883  733  412  798  656  955 1088  724]
`` Do n't chase me ! do n't chase me ! ''
9
[  97  886  883  101  195  412  906  565 1068  492]
When Boq saw her silver shoes he sa

#### Comments

* The paragraph in each mixture component having the highest membership probability doesn't match the corresponding top-10 words in the mixture component. This could be as a result of the fact that the paragraph might be near the center of the cluster since it is composed of generic words that may be common enough in every cluster. 
* More so, some paragraphs with the highest membership probability gave the same result.  

#### Exercise 4.2

In [344]:
# store the number of words in each paragraph
word_count = np.zeros((len(mydownloaded_lemmatizedtexts), 1))
for k in range(len(mydownloaded_lemmatizedtexts)):
    count = len(mydownloaded_lemmatizedtexts[k])
    word_count[k] = count
    #print(count)

In [346]:
longest_paragraphindex = np.argsort(-1*word_count, axis=0)[0]

In [353]:
# longest paragraph index and number of words in the paragraph
longest_paragraphindex, len(mydownloaded_lemmatizedtexts[584])

(array([584], dtype=int64), 233)

In [361]:
# longest paragraph
print(' '.join(mydownloaded_lemmatizedtexts[584]))

she leave dorothy alone and go back to the others . these she also lead to room , and each one of them find himself lodge in a very pleasant part of the palace . of course this politeness be waste on the scarecrow ; for when he find himself alone in his room he stand stupidly in one spot , just within the doorway , to wait till morning . it would not rest him to lie down , and he could not close his eye ; so he remain all night star at a little spider which be weave its web in a corner of the room , just as if it be not one of the most wonderful room in the world . the tin woodman lay down on his bed from force of habit , for he remember when he be make of flesh ; but not be able to sleep he pass the night move his joint up and down to make sure they keep in good work order . the lion would have prefer a bed of dried leaf in the forest , and do not like be shut up in a room ; but he have too much sense to let this worry him , so he spring upon the bed and roll himself up like a cat and